In [1]:
import json

with open('arxiv-metadata-oai-snapshot.json', 'r') as f:
    for i in range(5):  # 读取前5行
        line = f.readline()
        data = json.loads(line)
        print(json.dumps(data, indent=2))


{
  "id": "0704.0001",
  "submitter": "Pavel Nadolsky",
  "authors": "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  "title": "Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies",
  "comments": "37 pages, 15 figures; published version",
  "journal-ref": "Phys.Rev.D76:013009,2007",
  "doi": "10.1103/PhysRevD.76.013009",
  "report-no": "ANL-HEP-PR-07-12",
  "categories": "hep-ph",
  "license": null,
  "abstract": "  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with

In [ ]:

# create db

# 连接 PostgreSQL 创建数据库
def create_database():
    try:
        # 连接到 PostgreSQL（不指定数据库名，默认连接到 postgres 数据库）
        conn = psycopg2.connect(
            dbname="postgres",
            user=USER,
            password=PASSWORD,
            host=HOST,
            port=PORT
        )
        conn.autocommit = True  # 设置为自动提交事务（创建数据库需要）
        cur = conn.cursor()

        # 创建数据库（如果不存在）
        cur.execute(sql.SQL("SELECT 1 FROM pg_catalog.pg_database WHERE datname = %s"), [DB_NAME])
        if not cur.fetchone():
            print(f"Database {DB_NAME} does not exist. Creating it...")
            cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(DB_NAME)))
            print(f"Database {DB_NAME} created successfully.")
        else:
            print(f"Database {DB_NAME} already exists.")
        
        # 创建数据库用户
        cur.execute(sql.SQL("SELECT 1 FROM pg_catalog.pg_roles WHERE rolname = %s"), [DB_USER])
        if not cur.fetchone():
            print(f"User {DB_USER} does not exist. Creating it...")
            cur.execute(sql.SQL("CREATE USER {} WITH PASSWORD %s").format(sql.Identifier(DB_USER)), [DB_USER_PASSWORD])
            print(f"User {DB_USER} created successfully.")
        else:
            print(f"User {DB_USER} already exists.")
        
        # 给用户权限
        cur.execute(sql.SQL("GRANT ALL PRIVILEGES ON DATABASE {} TO {}").format(sql.Identifier(DB_NAME), sql.Identifier(DB_USER)))

        # 关闭连接
        cur.close()
        conn.close()

        print(f"Database setup complete. User {DB_USER} has access to {DB_NAME}.")

    except Exception as e:
        print(f"Error: {e}")

# 连接到创建的数据库并创建表
def create_tables():
    try:
        # 连接到已创建的数据库
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_USER_PASSWORD,
            host=HOST,
            port=PORT
        )
        cur = conn.cursor()

        # 创建表
        cur.execute("""
            CREATE TABLE IF NOT EXISTS arxiv_papers (
                id TEXT PRIMARY KEY,
                submitter TEXT,
                authors TEXT,
                title TEXT,
                comments TEXT,
                journal_ref TEXT,
                doi TEXT,
                report_no TEXT,
                categories TEXT,
                license TEXT,
                abstract TEXT,
                update_date DATE
            );
        """)

        cur.execute("""
            CREATE TABLE IF NOT EXISTS arxiv_versions (
                paper_id TEXT REFERENCES arxiv_papers(id),
                version TEXT,
                created TIMESTAMP
            );
        """)

        cur.execute("""
            CREATE TABLE IF NOT EXISTS arxiv_authors_parsed (
                paper_id TEXT REFERENCES arxiv_papers(id),
                last_name TEXT,
                first_name TEXT,
                middle_name TEXT
            );
        """)

        cur.execute("""
            ALTER TABLE arxiv_papers ADD COLUMN IF NOT EXISTS search_vector tsvector;
        """)

        cur.execute("""
            CREATE INDEX IF NOT EXISTS idx_arxiv_search_vector ON arxiv_papers USING GIN(search_vector);
        """)

        # 提交更改并关闭连接
        conn.commit()
        cur.close()
        conn.close()

        print("Tables created successfully.")

    except Exception as e:
        print(f"Error: {e}")

# 主程序
if __name__ == "__main__":
    create_database()
    create_tables()


Database arxiv_db already exists.
User postgres already exists.
Database setup complete. User postgres has access to arxiv_db.
Tables created successfully.


In [5]:
try:
    # Reconnect to the database
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_USER_PASSWORD,
        host=HOST,
        port=PORT
    )
    cur = conn.cursor()
    
    # Query basic information about the database
    cur.execute("""
        SELECT 
            datname AS database_name,
            numbackends AS active_connections,
            xact_commit AS transactions_committed,
            xact_rollback AS transactions_rolled_back,
            blks_read AS blocks_read,
            blks_hit AS blocks_hit
        FROM pg_stat_database
        WHERE datname = %s;
    """, (DB_NAME,))
    
    # Fetch and print the result
    db_info = cur.fetchone()
    print("Database Information:")
    print(f"Name: {db_info[0]}")
    print(f"Active Connections: {db_info[1]}")
    print(f"Transactions Committed: {db_info[2]}")
    print(f"Transactions Rolled Back: {db_info[3]}")
    print(f"Blocks Read: {db_info[4]}")
    print(f"Blocks Hit: {db_info[5]}")

    # Close the connection
    cur.close()
    conn.close()

except Exception as e:
    print(f"Error retrieving database information: {e}")

Database Information:
Name: arxiv_db
Active Connections: 1
Transactions Committed: 12
Transactions Rolled Back: 1
Blocks Read: 0
Blocks Hit: 3373


In [ ]:
#insert json into db

from tqdm import tqdm
import json
from datetime import datetime

try:
    # Reconnect to the database
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_USER_PASSWORD,
        host=HOST,
        port=PORT
    )
    cur = conn.cursor()

    # Open the JSON file
    with open('arxiv-metadata-oai-snapshot.json', 'r') as f:
        # Get the total number of lines in the file
        total_lines = sum(1 for _ in open('arxiv-metadata-oai-snapshot.json', 'r'))
        f.seek(0)  # Reset file pointer to the beginning

        batch_size = 1000
        batch = []

        # Use tqdm to display the progress bar
        with tqdm(total=total_lines, desc="Importing JSON data") as pbar:
            for line in f:
                # Parse each line as JSON
                data = json.loads(line)

                # Extract relevant fields
                paper_id = data.get('id')
                submitter = data.get('submitter')
                authors = data.get('authors')
                title = data.get('title')
                comments = data.get('comments')
                journal_ref = data.get('journal-ref')
                doi = data.get('doi')
                report_no = data.get('report-no')
                categories = data.get('categories')
                license = data.get('license')
                abstract = data.get('abstract')
                update_date = data.get('update_date')

                # Convert update_date to a proper date format
                if update_date:
                    update_date = datetime.strptime(update_date, '%Y-%m-%d').date()

                # Add to batch
                batch.append((
                    paper_id, submitter, authors, title, comments, journal_ref, doi,
                    report_no, categories, license, abstract, update_date
                ))

                # Insert batch into the database
                if len(batch) >= batch_size:
                    cur.executemany("""
                        INSERT INTO arxiv_papers (
                            id, submitter, authors, title, comments, journal_ref, doi,
                            report_no, categories, license, abstract, update_date
                        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        ON CONFLICT (id) DO NOTHING;
                    """, batch)
                    conn.commit()
                    batch = []

                # Update the progress bar
                pbar.update(1)

            # Insert any remaining records
            if batch:
                cur.executemany("""
                    INSERT INTO arxiv_papers (
                        id, submitter, authors, title, comments, journal_ref, doi,
                        report_no, categories, license, abstract, update_date
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT (id) DO NOTHING;
                """, batch)
                conn.commit()

    print("Data successfully inserted into the database.")

    # Close the connection
    cur.close()
    conn.close()

except Exception as e:
    print(f"Error inserting data into the database: {e}")

Data successfully inserted into the database.


In [8]:
# db info

import psycopg2
from psycopg2 import sql

# 数据库连接参数
HOST = 'localhost'  # 数据库主机
PORT = '5432'  # PostgreSQL 默认端口
USER = 'postgres'  # 用于创建数据库的管理员用户名
PASSWORD = '123456'  # 管理员密码
DB_NAME = 'arxiv_db'  # 要创建的数据库名称
DB_USER = 'postgres'  # 新用户
DB_USER_PASSWORD = '123456'  # 新用户密码
try:
    # Connect to the PostgreSQL server
    conn = psycopg2.connect(
        dbname="postgres",  # 连接到默认的 postgres 数据库
        user=USER,
        password=PASSWORD,
        host=HOST,
        port=PORT
    )
    cur = conn.cursor()

    # Query all databases
    cur.execute("SELECT datname FROM pg_database WHERE datistemplate = false;")
    databases = cur.fetchall()

    # Print the list of databases
    print("Available Databases:")
    for db in databases:
        print(f"- {db[0]}")

    # Close the connection
    cur.close()
    conn.close()

except Exception as e:
    print(f"Error querying databases: {e}")
try:
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_USER_PASSWORD,
        host=HOST,
        port=PORT
    )
    cur = conn.cursor()

    # 1. 基本数据库信息
    print("="*50)
    print("数据库基本信息:")
    cur.execute("""
        SELECT 
            datname AS database_name,
            pg_size_pretty(pg_database_size(datname)) AS database_size,
            numbackends AS active_connections,
            xact_commit AS transactions_committed,
            xact_rollback AS transactions_rolled_back,
            blks_read AS blocks_read,
            blks_hit AS blocks_hit
        FROM pg_stat_database
        WHERE datname = %s;
    """, (DB_NAME,))
    db_info = cur.fetchone()
    print(f"数据库名称: {db_info[0]}")
    print(f"数据库大小: {db_info[1]}")
    print(f"活动连接数: {db_info[2]}")
    print(f"已提交事务: {db_info[3]}")
    print(f"回滚事务: {db_info[4]}")
    print(f"磁盘读取块: {db_info[5]}")
    print(f"缓存命中块: {db_info[6]}")
    # 2. 表信息
    print("\n"+"="*50)
    print("表信息:")
    cur.execute("""
        SELECT 
            relname AS table_name,
            n_live_tup AS row_count,
            pg_size_pretty(pg_total_relation_size(quote_ident(relname))) AS total_size,
            pg_size_pretty(pg_table_size(quote_ident(relname))) AS table_size,
            pg_size_pretty(pg_indexes_size(quote_ident(relname))) AS index_size
        FROM pg_stat_user_tables
        ORDER BY n_live_tup DESC;
    """)
    tables_info = cur.fetchall()
    for table in tables_info:
        print(f"\n表名: {table[0]}")
        print(f"行数: {table[1]}")
        print(f"总大小: {table[2]}")
        print(f"表大小: {table[3]}")
        print(f"索引大小: {table[4]}")
    # 3. 表结构信息
    print("\n"+"="*50)
    print("表结构信息:")
    for table_info in tables_info:
        table_name = table_info[0]
        print(f"\n表 {table_name} 的结构:")
        cur.execute("""
            SELECT 
                column_name,
                data_type,
                character_maximum_length,
                column_default,
                is_nullable
            FROM information_schema.columns
            WHERE table_name = %s;
        """, (table_name,))
        columns = cur.fetchall()
        for col in columns:
            print(f"列名: {col[0]}")
            print(f"数据类型: {col[1]}")
            print(f"最大长度: {col[2]}")
            print(f"默认值: {col[3]}")
            print(f"允许空值: {col[4]}")
            print("-"*30)

    # 4. 索引信息
    print("\n"+"="*50)
    print("索引信息:")
    cur.execute("""
        SELECT
            schemaname,
            tablename,
            indexname,
            indexdef
        FROM pg_indexes
        WHERE schemaname = 'public'
        ORDER BY tablename, indexname;
    """)
    indexes = cur.fetchall()
    for idx in indexes:
        print(f"\n模式: {idx[0]}")
        print(f"表名: {idx[1]}")
        print(f"索引名: {idx[2]}")
        print(f"索引定义: {idx[3]}")
    cur.close()
    conn.close()


except Exception as e:
    print(f"Error inserting data into the database: {e}")

Available Databases:
- postgres
- arxiv_db
数据库基本信息:
数据库名称: arxiv_db
数据库大小: 3599 MB
活动连接数: 2
已提交事务: 1605
回滚事务: 5
磁盘读取块: 451
缓存命中块: 53733

表信息:

表名: arxiv_authors_parsed
行数: 0
总大小: 8192 bytes
表大小: 8192 bytes
索引大小: 0 bytes

表名: arxiv_versions
行数: 0
总大小: 8192 bytes
表大小: 8192 bytes
索引大小: 0 bytes

表名: arxiv_papers
行数: 0
总大小: 3591 MB
表大小: 3500 MB
索引大小: 91 MB

表结构信息:

表 arxiv_authors_parsed 的结构:
列名: paper_id
数据类型: text
最大长度: None
默认值: None
允许空值: YES
------------------------------
列名: last_name
数据类型: text
最大长度: None
默认值: None
允许空值: YES
------------------------------
列名: first_name
数据类型: text
最大长度: None
默认值: None
允许空值: YES
------------------------------
列名: middle_name
数据类型: text
最大长度: None
默认值: None
允许空值: YES
------------------------------

表 arxiv_versions 的结构:
列名: created
数据类型: timestamp without time zone
最大长度: None
默认值: None
允许空值: YES
------------------------------
列名: paper_id
数据类型: text
最大长度: None
默认值: None
允许空值: YES
------------------------------
列名: version
数据类型: text
最大长度: None
默认值: Non

In [9]:
# 数据库连接参数
HOST = 'localhost'  # 数据库主机
PORT = '5432'  # PostgreSQL 默认端口
USER = 'postgres'  # 用于创建数据库的管理员用户名
PASSWORD = '123456'  # 管理员密码
DB_NAME = 'arxiv_db'  # 要创建的数据库名称
DB_USER = 'postgres'  # 新用户
DB_USER_PASSWORD = '123456'  # 新用户密码

try:
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_USER_PASSWORD,
        host=HOST,
        port=PORT
    )
    cur = conn.cursor()

    print("真实行数统计：")
    for table_info in tables_info:
        table_name = table_info[0]
        cur.execute(f"SELECT COUNT(*) FROM {table_name};")
        real_count = cur.fetchone()[0]
        print(f"表 {table_name} 实际行数: {real_count}")

    cur.close()
    conn.close()
except Exception as e:
    print(f"Error counting rows: {e}")

真实行数统计：
表 arxiv_authors_parsed 实际行数: 0
表 arxiv_versions 实际行数: 0
表 arxiv_papers 实际行数: 2725378


######ANalyse abstract###########
# Aim: classification
# 1 Read the abstract and catagory from db,  
# 2 

use the abstract to classify all articles, since the catagory tag in the db already specify the 1st level of tags, i need to use openAI type model to do further claasification based on the abstract. there is a local model api that can be called using localhost:

In [ ]:
#

In [ ]:
#verterize into LLM



